In [1]:
import pandas as pd

df = pd.read_excel(
    "Pedidos.xlsx",
    parse_dates=["DataFechamento"]  # Converte diretamente essa coluna para datetime
)
df.head()

# pega todas as colunas numéricas
num_cols = df.select_dtypes(include="number").columns

# remove qtde_vendida da lista
num_cols = num_cols.drop("qtde_vendida")

# arredonda apenas as numéricas restantes
df[num_cols] = df[num_cols].round(2)

df.head()


# Ver linhas com valores negativos
linhas_removidas = (df["qtde_vendida"] < 0).sum()
print("Linhas a remover:", linhas_removidas)

df = df[df["qtde_vendida"] >= 0].copy()  

df = df[df["DataFechamento"].dt.year < 2025].copy()

df = df.drop_duplicates()

df["valor_calculado"] = df["qtde_vendida"] * df["valor_unitario"] + df["acrescimos"] - df["descontos"]

# Diferença entre o calculado e o informado
df["diferenca"] = (df["valor_total"] - df["valor_calculado"]).round(2)

# Conferir registros inconsistentes
inconsistentes = df[df["diferenca"] != 0]

from scipy import stats

# Z-score da coluna 'valor_pedido'
z_scores = stats.zscore(df['valor_pedido'])
df_sem_outliers = df[(z_scores > -3) & (z_scores < 3)]


Linhas a remover: 14


In [ ]:
# ------------------------------
# 📌 Preparação dos dados para o modelo (com lags + tqdm)
# ------------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm import tqdm
import numpy as np

# 🔹 Defina sua variável alvo (target)
target = "valor_total"
X = df_sem_outliers.drop(columns=[target])
y = df_sem_outliers[target]

# ------------------------------
# 🔹 Engenharia de atributos
# ------------------------------
# Datas → extrair dia, mês, ano e dia da semana
X["ano"] = df_sem_outliers["DataFechamento"].dt.year
X["mes"] = df_sem_outliers["DataFechamento"].dt.month
X["dia_semana"] = df_sem_outliers["DataFechamento"].dt.dayofweek

# Ticket médio
X["ticket_medio"] = (
    df_sem_outliers["valor_total"] / df_sem_outliers["qtde_vendida"]
).replace([np.inf, -np.inf], 0).fillna(0)

# 🔹 Criar lags para qtde_vendida
for lag in range(1, 4):  # lag_1, lag_2, lag_3
    X[f"lag_{lag}"] = df_sem_outliers["qtde_vendida"].shift(lag)

# Remover linhas iniciais com NaN causadas pelos lags
mask = X[[f"lag_{lag}" for lag in range(1, 4)]].notnull().all(axis=1)
X = X[mask]
y = y[mask]

# ------------------------------
# 🔹 Divisão treino/teste
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False  # sem embaralhar por ser série temporal
)

# ------------------------------
# 🔹 Pré-processamento
# ------------------------------
num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X_train.select_dtypes(include=["object", "category"]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

# ------------------------------
# 🔹 Pipeline do modelo com tqdm
# ------------------------------
# ProgressBar só no fit (RandomForest não tem callback), usamos tqdm "manual"
with tqdm(total=1, desc="Treinando modelo") as pbar:
    model = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))
    ])
    model.fit(X_train, y_train)
    pbar.update(1)

# ------------------------------
# 🔹 Avaliação
# ------------------------------
y_pred = model.predict(X_test)




Treinando modelo: 100%|██████████| 1/1 [15:09<00:00, 909.80s/it]


AttributeError: 'float' object has no attribute 'round'

In [9]:
print("MAE:", round(mean_absolute_error(y_test, y_pred), 2))
print("RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred)), 2))  # raiz do MSE
print("R²:", round(r2_score(y_test, y_pred), 2))


MAE: 0.02
RMSE: 0.2
R²: 1.0
